# TensorFlow avec Horovod

<img src='https://github.com/retkowsky/images/blob/master/AzureMLservicebanniere.png?raw=true'>

The TensorFlow estimator also supports distributed training across CPU and GPU clusters. You can easily run distributed TensorFlow jobs and Azure Machine Learning will manage the orchestration for you.

Azure Machine Learning supports two methods of distributed training in TensorFlow:

- **MPI-based distributed training** using the Horovod framework<br>
- **Native distributed TensorFlow** using the parameter server method

**Horovod**
Horovod is an open-source framework for distributed training developed by Uber. It offers an easy path to distributed GPU TensorFlow jobs.

To use Horovod, specify an **MpiConfiguration object** for the distributed_training parameter in the TensorFlow constructor. This parameter ensures that Horovod library is installed for you to use in your training script.

## 1. Infos

In [ ]:
import sys
sys.version

In [ ]:
import datetime
maintenant = datetime.datetime.now()
print('Date :', maintenant)

In [ ]:
import azureml.core
print("Azure ML version :", azureml.core.VERSION)

In [ ]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Resource group: ' + ws.resource_group, sep = '\n')

## 2. Création compute GPU

In [ ]:
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, " - " , ct.type, " - ", ct.provisioning_state)

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "gpuclusterNC6"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           min_nodes=1,
                                                           max_nodes=8)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)


In [ ]:
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, " - " , ct.type, " - ", ct.provisioning_state)

## 3. Chargement des données

In [ ]:
import os
import urllib

os.makedirs('./data', exist_ok=True)
download_url = 'http://mattmahoney.net/dc/text8.zip'
urllib.request.urlretrieve(download_url, filename='./data/text8.zip')

In [ ]:
ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)

In [ ]:
ds.upload(src_dir='data', target_path='data', overwrite=True, show_progress=True)

In [ ]:
path_on_datastore = 'data/text8.zip'
ds_data = ds.path(path_on_datastore)
print(ds_data)

## 4. Apprentissage du modèle

In [ ]:
import os

project_folder = './tf-distr-hvd'
os.makedirs(project_folder, exist_ok=True)

In [ ]:
import shutil

shutil.copy('tf_horovod_word2vec.py', project_folder)

In [ ]:
from azureml.core import Experiment

experiment_name = 'Exemple13-TFHorovod'
experiment = Experiment(ws, name=experiment_name)

### 4.1 Estimateur TensorFlow
The AML SDK's TensorFlow estimator enables you to easily submit TensorFlow training jobs for both single-node and distributed runs. For more information on the TensorFlow estimator, refer [here](https://docs.microsoft.com/azure/machine-learning/service/how-to-train-tensorflow).

In [ ]:
from azureml.train.dnn import TensorFlow

script_params={
    '--input_data': ds_data
}

estimator= TensorFlow(source_directory=project_folder,
                      compute_target=compute_target,
                      script_params=script_params,
                      entry_script='tf_horovod_word2vec.py',
                      node_count=2,
                      process_count_per_node=1,
                      distributed_training='mpi', #Pour horovod
                      use_gpu=True)

The above code specifies that we will run our training script on `2` nodes, with one worker per node. In order to execute a distributed run using MPI/Horovod, you must provide the argument `distributed_backend='mpi'`. Using this estimator with these settings, TensorFlow, Horovod and their dependencies will be installed for you. However, if your script also uses other packages, make sure to install them via the `TensorFlow` constructor's `pip_packages` or `conda_packages` parameters.

Note that we passed our training data reference `ds_data` to our script's `--input_data` argument. This will 1) mount our datastore on the remote compute and 2) provide the path to the data zip file on our datastore.

### 4.2 Run

> Nécessite 10 minutes de temps de traitement

In [ ]:
tags = {"Type": "test" , "Langage" : "Python" , "Framework" : "Tensorflow Horovod"}

In [ ]:
run = experiment.submit(estimator, tags=tags)
print(run)

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()

In [ ]:
# Progression du run
run.get_details()

In [ ]:
# Statut
compute_target.list_nodes()

## 4.3 Résultats

In [ ]:
run.wait_for_completion(show_output=True)

In [ ]:
run.get_metrics()

In [ ]:
experiment

## 5. Suppression ressource

In [ ]:
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, " - " , ct.type, " - ", ct.provisioning_state)

In [ ]:
#Pour supprimer le compute server
#compute_target.delete()

In [ ]:
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, " - " , ct.type, " - ", ct.provisioning_state)

<img src="https://github.com/retkowsky/images/blob/master/Powered-by-MS-Azure-logo-v2.png?raw=true" height="300" width="300">